In [1]:
%matplotlib inline


Creating extensions using numpy and scipy
=========================================
**Author**: `Adam Paszke <https://github.com/apaszke>`_

In this tutorial, we shall go through two tasks:

1. Create a neural network layer with no parameters.

    -  This calls into **numpy** as part of it’s implementation

2. Create a neural network layer that has learnable weights

    -  This calls into **SciPy** as part of it’s implementation



In [2]:
import torch
from torch.autograd import Function
from torch.autograd import Variable

Parameter-less example
----------------------

This layer doesn’t particularly do anything useful or mathematically
correct.

It is aptly named BadFFTFunction

**Layer Implementation**



In [3]:
from numpy.fft import rfft2, irfft2


class BadFFTFunction(Function):

    def forward(self, input):
        numpy_input = input.numpy()
        result = abs(rfft2(numpy_input))
        return torch.FloatTensor(result)

    def backward(self, grad_output):
        numpy_go = grad_output.numpy()
        result = irfft2(numpy_go)
        return torch.FloatTensor(result)

# since this layer does not have any parameters, we can
# simply declare this as a function, rather than as an nn.Module class


def incorrect_fft(input):
    return BadFFTFunction()(input)

**Example usage of the created layer:**



In [5]:
input = Variable(torch.randn(8, 8), requires_grad=True)
result = incorrect_fft(input)
print(result.data)
result.backward(torch.randn(result.size()))
print(input.grad)


  6.1829  11.9744   9.8176  10.8280   0.5950
  4.8383   5.6456  10.3967   2.3743   1.9450
  6.2943   6.1773   5.7614   6.7952   2.6731
  7.7006   3.2616  13.8528   7.4106  11.7680
  1.0098  19.0406   5.7653   0.6019   2.0653
  7.7006   2.5807   8.4803  11.8092  11.7680
  6.2943   3.4272   8.0550   8.7194   2.6731
  4.8383   6.6792   9.6886   8.8778   1.9450
[torch.FloatTensor of size 8x5]

Variable containing:
 0.0523  0.0843 -0.0460  0.0436 -0.0750  0.0436 -0.0460  0.0843
 0.0413 -0.0303 -0.0125  0.2037 -0.0425 -0.0145  0.0988  0.0422
-0.0930 -0.1588 -0.0845  0.1245  0.2047  0.1530 -0.0745 -0.0850
 0.2796 -0.1453 -0.2112 -0.1489  0.0792  0.0275 -0.0544 -0.0301
 0.0008  0.0009  0.0486 -0.1282 -0.1561 -0.1282  0.0486  0.0009
 0.2796 -0.0301 -0.0544  0.0275  0.0792 -0.1489 -0.2112 -0.1453
-0.0930 -0.0850 -0.0745  0.1530  0.2047  0.1245 -0.0845 -0.1588
 0.0413  0.0422  0.0988 -0.0145 -0.0425  0.2037 -0.0125 -0.0303
[torch.FloatTensor of size 8x8]



Parametrized example
--------------------

This implements a layer with learnable weights.

It implements the Cross-correlation with a learnable kernel.

In deep learning literature, it’s confusingly referred to as
Convolution.

The backward computes the gradients wrt the input and gradients wrt the
filter.

**Implementation:**

*Please Note that the implementation serves as an illustration, and we
did not verify it’s correctness*



In [6]:
from scipy.signal import convolve2d, correlate2d
from torch.nn.modules.module import Module
from torch.nn.parameter import Parameter


class ScipyConv2dFunction(Function):

    def forward(self, input, filter):
        result = correlate2d(input.numpy(), filter.numpy(), mode='valid')
        self.save_for_backward(input, filter)
        return torch.FloatTensor(result)

    def backward(self, grad_output):
        input, filter = self.saved_tensors
        grad_input = convolve2d(grad_output.numpy(), filter.t().numpy(), mode='full')
        grad_filter = convolve2d(input.numpy(), grad_output.numpy(), mode='valid')
        return torch.FloatTensor(grad_input), torch.FloatTensor(grad_filter)


class ScipyConv2d(Module):

    def __init__(self, kh, kw):
        super(ScipyConv2d, self).__init__()
        self.filter = Parameter(torch.randn(kh, kw))

    def forward(self, input):
        return ScipyConv2dFunction()(input, self.filter)

**Example usage:**



In [7]:
module = ScipyConv2d(3, 3)
print(list(module.parameters()))
input = Variable(torch.randn(10, 10), requires_grad=True)
output = module(input)
print(output)
output.backward(torch.randn(8, 8))
print(input.grad)

[Parameter containing:
-2.6030 -0.3843  0.5587
 0.2654  1.3264 -2.1400
 1.9451  0.0599 -0.2262
[torch.FloatTensor of size 3x3]
]
Variable containing:
  3.5203  -1.3729   3.1299   3.4696  -0.2412   3.3267   4.0775 -11.2150
  4.1776  -2.4674   2.0483   0.0354   1.3668   6.8117  -4.1741   1.5168
 -5.6433  -2.6592   2.3315  -0.3444  -2.2906  -2.6987  -4.0895   2.3890
 -5.0757   1.4878   3.6190  -3.1724   2.4341   1.4937  -3.1806  -8.2635
 -1.5748   1.8645  -2.3110  -1.3063  -2.5158   5.8617  -2.5098   1.9802
  2.2826   4.3955   0.5357   1.6774   3.4699  -6.2313  -5.0198  10.3553
  4.9095   0.0631  -5.9529   7.2286   5.4995  -6.0974   0.7383   2.1105
 -4.5536  -6.7273   9.5790  -2.9598  -6.3236   1.5818   5.8783  -2.4652
[torch.FloatTensor of size 8x8]

Variable containing:

Columns 0 to 7 
 -1.1491   3.1455  -6.2042  -2.1850   4.7667   0.2857   3.5723   3.6078
  3.7457   1.8007  -6.4104   4.5249  -0.3350  -0.4320   7.2227  -3.8357
 -5.0664   0.4539   8.9772  -4.8787  -1.3874  -1.8264  -4.6